In [1]:

from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
import numpy as np
from keras.applications.nasnet import NASNetLarge
from keras.applications.nasnet import NASNetMobile

from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.optimizers import SGD


Using TensorFlow backend.


In [ ]:
#VGG19

#base_model = VGG19(weights='imagenet')
base_model = NASNetMobile(weights = 'weights/NASNet-mobile.h5')

#input_shape=None, include_top=True, weights='imagenet', input_tensor=None, pooling=None, classes=1000

model = Model(inputs=base_model.input, outputs=base_model.get_layer('block4_pool').output)
 
img_path = 'test_data/tiger.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
 
block4_pool_features = model.predict(x)

In [ ]:
block4_pool_features

In [ ]:
# inceptionv3
 
# 构建不带分类器的预训练模型
base_model = InceptionV3(weights='imagenet', include_top=False)

# 添加全局平均池化层
x = base_model.output
x = GlobalAveragePooling2D()(x)
 
# 添加一个全连接层
x = Dense(1024, activation='relu')(x)
 
# 添加一个分类器，假设我们有200个类
predictions = Dense(200, activation='softmax')(x)
 
# 构建我们需要训练的完整模型
model = Model(inputs=base_model.input, outputs=predictions)
 
# 首先，我们只训练顶部的几层（随机初始化的层）
# 锁住所有 InceptionV3 的卷积层
for layer in base_model.layers:
    layer.trainable = False
 
#编译模型（一定要在锁层以后操作）
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
 
# 在新的数据集上训练几代
model.fit_generator(...)
 
# 现在顶层应该训练好了，让我们开始微调 Inception V3 的卷积层。
# 我们会锁住底下的几层，然后训练其余的顶层。
 
# 让我们看看每一层的名字和层号，看看我们应该锁多少层呢：
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)
 
# 我们选择训练最上面的两个 Inception block
# 也就是说锁住前面249层，然后放开之后的层。
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True
 
# 我们需要重新编译模型，才能使上面的修改生效
# 让我们设置一个很低的学习率，使用 SGD 来微调
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')
 
# 我们继续训练模型，这次我们训练最后两个 Inception block
# 和两个全连接层
model.fit_generator(...)

In [ ]:
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input
 
# 这也可能是不同的 Keras 模型或层的输出
input_tensor = Input(shape=(224, 224, 3))  # 假定 K.image_data_format() == 'channels_last'
 
model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=True)

In [2]:
#NASNetLarge(input_shape=None, include_top=True, weights='imagenet', input_tensor=None, pooling=None, classes=1000)
#NASNetMobile(input_shape=None, include_top=True, weights='imagenet', input_tensor=None, pooling=None, classes=1000)
# 构建不带分类器的预训练模型
base_model = NASNetMobile(weights='weights/NASNet-mobile.h5', include_top=False)

# 添加全局平均池化层
x = base_model.output
x = GlobalAveragePooling2D()(x)
 
# 添加一个全连接层
x = Dense(1024, activation='relu')(x)
 
# 添加一个分类器，假设我们有200个类
predictions = Dense(257, activation='softmax')(x)

# 构建我们需要训练的完整模型
model = Model(inputs=base_model.input, outputs=predictions)
 
# 首先，我们只训练顶部的几层（随机初始化的层）
# 锁住所有 InceptionV3 的卷积层
for layer in base_model.layers:
    layer.trainable = False

#编译模型（一定要在锁层以后操作）
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
 
# 在新的数据集上训练几代
model.fit_generator(...)
 
# 现在顶层应该训练好了，让我们开始微调 Inception V3 的卷积层。
# 我们会锁住底下的几层，然后训练其余的顶层。
 
# 让我们看看每一层的名字和层号，看看我们应该锁多少层呢：
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)
# 我们选择训练最上面的两个 Inception block
# 也就是说锁住前面249层，然后放开之后的层。
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True
 
# 我们需要重新编译模型，才能使上面的修改生效
# 让我们设置一个很低的学习率，使用 SGD 来微调

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')
 
# 我们继续训练模型，这次我们训练最后两个 Inception block
# 和两个全连接层
model.fit_generator(...)

Instructions for updating:
Colocations handled automatically by placer.


ValueError: You are trying to load a weight file containing 389 layers into a model with 388 layers.

In [8]:
base_model = NASNetMobile(weights='weights/NASNet-mobile.h5')#, include_top=False, pooling = None)
for layer in base_model.layers[:249]:
    print(layer)